In [320]:
import numpy as np
import pandas as pd
import lightgbm

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import scipy.sparse as sparse
import random
import implicit
from implicit.als import AlternatingLeastSquares as ALS

%cd /home/user_3/medistream-recsys/Script
from preprocessing import drop_columns,dict_to_column,dict_to_set,set_to_column,key_to_element

pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 100)

from dateutil.relativedelta import relativedelta
from datetime import datetime, timedelta
import scipy.sparse as sp
from implicit.lmf import LogisticMatrixFactorization as LMF

/home/user_3/medistream-recsys/Script


In [321]:
# products name 확인 용
products_df = pd.read_json("/fastcampus-data/products/products.json")
products_df = key_to_element(['_id'],products_df)
df = pd.read_json('/fastcampus-data/select_column_version_4.json')
df['date_paid'] = pd.to_datetime(df['date_paid'])

100%|██████████| 5141/5141 [00:00<00:00, 744678.71it/s]


In [322]:
df['date_paid'].max()-relativedelta(months=2)

Timestamp('2022-07-13 08:59:21.151000+0000', tz='UTC')

In [323]:
date_state = "2022-07-13"

# paid orders만 가져오기
df['date_paid'] = pd.to_datetime(df['date_paid'])
df_only_paid = df[~df['date_paid'].isna()]
# 2개월치 데이터만 가져오기
df_date = df_only_paid[df_only_paid['date_paid'] >= date_state]
# 취소 안된 것만 가져오기
complete_df = df_date[(df_date['paid'] == True) & (df_date['cancelled']==False)]
# 도서 카테고리만 가져오기
only_book = complete_df[complete_df['name'] == '도서']

# 유저가 중복으로 아이템 구매 삭제
df_duplicated_book = only_book.drop_duplicates(subset=['customer_id','product_ids'])
df_book = df_duplicated_book.sort_values(by='date_paid').reset_index(drop=True)

In [324]:
# none 값 확인하기
df_book.isna().sum()

_id                0
date_created       0
regular_price      0
sale_price         0
three_months       0
date_paid          0
customer_id        0
paid               0
name_x             0
category_id_y      0
product_ids        0
quantity           0
price              0
price_total        0
age_group        496
한의사 여부             0
사업자 여부             0
cancelled          0
name               0
slug               0
dtype: int64

In [421]:
len(df_book)

3227

In [420]:
len(set(df_book.customer_id))*5

9350

In [325]:
df_cat = df_book.copy()
df_cat['user_id'] = df_cat['customer_id'].astype("category").cat.codes
df_cat['item_id'] = df_cat['product_ids'].astype("category").cat.codes

item_lookup = df_cat[['item_id', 'product_ids','regular_price']].drop_duplicates()
# item_lookup['item_id'] = item_lookup.item_id.astype(str)

user_lookup = df_cat[['user_id', 'customer_id','한의사 여부','사업자 여부']].drop_duplicates()
# user_lookup['user_id'] = user_lookup.user_id.astype(str)

In [326]:
rows = df_cat['user_id'].astype(int)
cols = df_cat['item_id'].astype(int)

In [327]:
user_lookup.user_id.unique()

array([ 927, 1632,  947, ..., 1701,  807, 1702], dtype=int16)

In [328]:
users = list(np.sort(df_cat.user_id.unique()))
items = list(np.sort(df_cat.item_id.unique()))

In [329]:
mp = df_cat.groupby(['product_ids']).count()['customer_id'].reset_index().rename(columns={'customer_id':'cnt'})
mp_sort = mp.sort_values(by='cnt', ascending=False, ignore_index=True)
mp_lookup = mp_sort.head(30)

In [330]:
####################################################

In [331]:
from datetime import datetime, timedelta
datetime(2022,9,13)-timedelta(days=21)

datetime.datetime(2022, 8, 23, 0, 0)

In [332]:
date = '2022-08-23'
train = df_cat[df_cat['date_paid'] < date]
test = df_cat[df_cat['date_paid'] >= date]

In [333]:
# df_feature_select = df_book[['customer_id','product_ids','regular_price','price','price_total','한의사 여부','사업자 여부']]

In [334]:
####################################################

In [335]:
def neg_sampling(df_cat):
    all_neg_list = {'item_id':[],'user_id':[]}
    for user in rows.unique():
        cnt = 0

        pos_items = list(df_cat[df_cat['user_id']==user]['item_id'].unique())
        while True:
            item = np.random.randint(len(items))
            if cnt == 5:
                break
            elif (item not in pos_items) & (item not in neg_list):
                cnt += 1
                all_neg_list['user_id'].append(user)    
                all_neg_list['item_id'].append(item)
                
    neg_df = pd.DataFrame(all_neg_list)
    return neg_df

In [417]:
train_pos_neg

,item_id,user_id,한의사 여부,사업자 여부,regular_price,relevance
0,58,0,0.0,1.0,30000.0,0
1,201,0,0.0,1.0,48000.0,0
2,175,0,0.0,1.0,30000.0,0
3,94,0,0.0,1.0,38000.0,0
4,14,0,0.0,1.0,25000.0,0
...,...,...,...,...,...,...
11593,41,1869,0.0,0.0,38000.0,0
11594,119,1869,0.0,0.0,25000.0,0
11595,87,1869,0.0,0.0,45000.0,0
11596,127,1869,0.0,0.0,50000.0,0


In [336]:
train_neg = neg_sampling(train)
test_neg = neg_sampling(test)

In [337]:
def pos_neg_merge(neg:pd.DataFrame(),pos:pd.DataFrame()):
    user_neg = pd.merge(neg, user_lookup, how='left', on='user_id')
    item_neg = pd.merge(user_neg, item_lookup, how='left', on='item_id')
    item_neg['relevance'] = 0
    
    pos_feature_select = pos[['item_id','product_ids','regular_price',\
                              'user_id','customer_id','한의사 여부','사업자 여부']]
    pos_feature_select['relevance'] = 1
    
    pos_neg_df = pd.concat([item_neg, pos_feature_select],ignore_index=True)
    pos_neg_df = pos_neg_df.drop(columns=['product_ids','customer_id'])
    return pos_neg_df

In [377]:
train_pos_neg = pos_neg_merge(train_neg, train).sort_values(by='user_id',ignore_index=True)
test_pos_neg = pos_neg_merge(test_neg, test).sort_values(by='user_id',ignore_index=True)

<ipython-input-337-89571b754794>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pos_feature_select['relevance'] = 1


In [378]:
train_groups = train_pos_neg.groupby("user_id")["user_id"].count().to_numpy()
test_groups = test_pos_neg.groupby("user_id")["user_id"].count().to_numpy()

In [392]:
def one_hot_encoding(pos_neg_df:pd.DataFrame()):
    dummies_df = pd.get_dummies(pos_neg_df,columns=['user_id','item_id'])
    return dummies_df

In [393]:
lgb_train = one_hot_encoding(train_pos_neg)
lgb_test = one_hot_encoding(test_pos_neg)

In [395]:
def split_feature_target(lgb_df):
    x = lgb_df.drop(columns='relevance')
    y = lgb_df['relevance']
    return x,y

In [396]:
x_lgb_train, y_lgb_train = split_feature_target(lgb_train)
x_lgb_test, y_lgb_test = split_feature_target(lgb_test)

In [411]:
model = lightgbm.LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
    learning_rate=0.05,
    random_state=42,
    n_estimators=300,

)

In [416]:
model.fit(
    X=x_lgb_train,
    y=y_lgb_train,
    group= train_groups,
    eval_set=[(x_lgb_test, y_lgb_test)],
    eval_group=[test_groups],
    eval_at=15,
    verbose=10,
    early_stopping_rounds=None
)

[10]	valid_0's ndcg@15: 0.809594
[20]	valid_0's ndcg@15: 0.807845
[30]	valid_0's ndcg@15: 0.801812
[40]	valid_0's ndcg@15: 0.799109
[50]	valid_0's ndcg@15: 0.798816
[60]	valid_0's ndcg@15: 0.797283
[70]	valid_0's ndcg@15: 0.797786
[80]	valid_0's ndcg@15: 0.79737
[90]	valid_0's ndcg@15: 0.797615
[100]	valid_0's ndcg@15: 0.798327
[110]	valid_0's ndcg@15: 0.798572
[120]	valid_0's ndcg@15: 0.798603
[130]	valid_0's ndcg@15: 0.798725
[140]	valid_0's ndcg@15: 0.798401
[150]	valid_0's ndcg@15: 0.798185
[160]	valid_0's ndcg@15: 0.798585
[170]	valid_0's ndcg@15: 0.798713
[180]	valid_0's ndcg@15: 0.799033
[190]	valid_0's ndcg@15: 0.798397
[200]	valid_0's ndcg@15: 0.798188
[210]	valid_0's ndcg@15: 0.798038
[220]	valid_0's ndcg@15: 0.79845
[230]	valid_0's ndcg@15: 0.798544
[240]	valid_0's ndcg@15: 0.79878
[250]	valid_0's ndcg@15: 0.798637
[260]	valid_0's ndcg@15: 0.798586
[270]	valid_0's ndcg@15: 0.798459
[280]	valid_0's ndcg@15: 0.79863
[290]	valid_0's ndcg@15: 0.798336
[300]	valid_0's ndcg@15: 0.

LGBMRanker(learning_rate=0.05, metric='ndcg', n_estimators=300,
           objective='lambdarank', random_state=42)

In [353]:
lgb_train

,한의사 여부,사업자 여부,regular_price,relevance,user_id_0,user_id_1,user_id_2,user_id_3,user_id_4,user_id_5,user_id_6,user_id_7,user_id_8,user_id_9,user_id_10,user_id_11,user_id_12,user_id_13,user_id_14,user_id_15,user_id_16,user_id_17,user_id_18,user_id_19,user_id_20,user_id_21,user_id_22,user_id_23,user_id_24,user_id_25,user_id_26,user_id_27,user_id_28,user_id_29,user_id_30,user_id_31,user_id_32,user_id_33,user_id_34,user_id_35,user_id_36,user_id_37,user_id_38,user_id_39,user_id_40,user_id_41,user_id_42,user_id_43,user_id_44,user_id_45,...,item_id_154,item_id_155,item_id_156,item_id_157,item_id_158,item_id_159,item_id_160,item_id_161,item_id_162,item_id_163,item_id_164,item_id_165,item_id_166,item_id_167,item_id_168,item_id_169,item_id_170,item_id_171,item_id_172,item_id_173,item_id_174,item_id_175,item_id_176,item_id_177,item_id_178,item_id_179,item_id_180,item_id_181,item_id_182,item_id_183,item_id_184,item_id_185,item_id_186,item_id_187,item_id_188,item_id_189,item_id_190,item_id_191,item_id_192,item_id_193,item_id_194,item_id_195,item_id_196,item_id_197,item_id_198,item_id_199,item_id_200,item_id_201,item_id_202,item_id_203
0,1.0,0.0,34000.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,0.0,70000.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.0,0.0,15000.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.0,0.0,50000.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,0.0,55000.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11593,1.0,0.0,40000.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
11594,1.0,0.0,60000.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
11595,0.0,0.0,60000.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
11596,0.0,0.0,20000.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [365]:
len(train_groups)

1870

In [369]:
len(set(train_pos_neg.user_id))

1870

In [373]:
len(set(test_pos_neg.user_id))

1870

In [374]:
set(train_pos_neg.user_id) - set(test_pos_neg.user_id)

set()

In [375]:
set(test_pos_neg.user_id)- set(train_pos_neg.user_id)

set()

In [385]:
x_lgb_test

,한의사 여부,사업자 여부,regular_price,user_id_0,user_id_1,user_id_2,user_id_3,user_id_4,user_id_5,user_id_6,user_id_7,user_id_8,user_id_9,user_id_10,user_id_11,user_id_12,user_id_13,user_id_14,user_id_15,user_id_16,user_id_17,user_id_18,user_id_19,user_id_20,user_id_21,user_id_22,user_id_23,user_id_24,user_id_25,user_id_26,user_id_27,user_id_28,user_id_29,user_id_30,user_id_31,user_id_32,user_id_33,user_id_34,user_id_35,user_id_36,user_id_37,user_id_38,user_id_39,user_id_40,user_id_41,user_id_42,user_id_43,user_id_44,user_id_45,user_id_46,...,item_id_154,item_id_155,item_id_156,item_id_157,item_id_158,item_id_159,item_id_160,item_id_161,item_id_162,item_id_163,item_id_164,item_id_165,item_id_166,item_id_167,item_id_168,item_id_169,item_id_170,item_id_171,item_id_172,item_id_173,item_id_174,item_id_175,item_id_176,item_id_177,item_id_178,item_id_179,item_id_180,item_id_181,item_id_182,item_id_183,item_id_184,item_id_185,item_id_186,item_id_187,item_id_188,item_id_189,item_id_190,item_id_191,item_id_192,item_id_193,item_id_194,item_id_195,item_id_196,item_id_197,item_id_198,item_id_199,item_id_200,item_id_201,item_id_202,item_id_203
0,0.0,1.0,30000.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0.0,1.0,48000.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,0.0,1.0,30000.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0.0,1.0,38000.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0.0,1.0,25000.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11593,0.0,0.0,38000.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
11594,0.0,0.0,25000.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
11595,0.0,0.0,45000.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
11596,0.0,0.0,50000.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [386]:
x_lgb_train

,한의사 여부,사업자 여부,regular_price,user_id_0,user_id_1,user_id_2,user_id_3,user_id_4,user_id_5,user_id_6,user_id_7,user_id_8,user_id_9,user_id_10,user_id_11,user_id_12,user_id_13,user_id_14,user_id_15,user_id_16,user_id_17,user_id_18,user_id_19,user_id_20,user_id_21,user_id_22,user_id_23,user_id_24,user_id_25,user_id_26,user_id_27,user_id_28,user_id_29,user_id_30,user_id_31,user_id_32,user_id_33,user_id_34,user_id_35,user_id_36,user_id_37,user_id_38,user_id_39,user_id_40,user_id_41,user_id_42,user_id_43,user_id_44,user_id_45,user_id_46,...,item_id_154,item_id_155,item_id_156,item_id_157,item_id_158,item_id_159,item_id_160,item_id_161,item_id_162,item_id_163,item_id_164,item_id_165,item_id_166,item_id_167,item_id_168,item_id_169,item_id_170,item_id_171,item_id_172,item_id_173,item_id_174,item_id_175,item_id_176,item_id_177,item_id_178,item_id_179,item_id_180,item_id_181,item_id_182,item_id_183,item_id_184,item_id_185,item_id_186,item_id_187,item_id_188,item_id_189,item_id_190,item_id_191,item_id_192,item_id_193,item_id_194,item_id_195,item_id_196,item_id_197,item_id_198,item_id_199,item_id_200,item_id_201,item_id_202,item_id_203
0,0.0,1.0,30000.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0.0,1.0,48000.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,0.0,1.0,30000.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0.0,1.0,38000.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0.0,1.0,25000.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11593,0.0,0.0,38000.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
11594,0.0,0.0,25000.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
11595,0.0,0.0,45000.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
11596,0.0,0.0,50000.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
